<h4>(1 - Recommend). Create and save new notebook(s) to enable automatic re-mounting of Drive storage. Copy and paste the below code into a <a href="http://colab.research.google.com/#create=true" target="_parent">new notebook</a> in Colab and follow the directions in the <a href="../#how-to-use" target="_parent">How To Use</a> section of the <em>README</em></h4><h4>(2). Open the pre-formed version in Colab (requires manual authorization each time a notebook is opened) <a href="https://colab.research.google.com/github/tdulcet/Distributed-Computing-Scripts/blob/master/google-colab/GoogleColabGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h4>

In [ ]:
#@title # **📔 Colab GPU and CPU Notebook**#{ vertical-output: true, form-width: "96%", display-mode: "form" }

#@markdown #### ↖️ Click the ▶️ button after deciding on the options below.
#@markdown #### 🔌 Make sure the GPU is enabled under *Runtime→Change runtime type*
#@markdown #### 💡 Keep each notebook **open** in the browser to prevent disconnection. 📌 [the tab](https://support.mozilla.org/en-US/kb/pinned-tabs-keep-favorite-websites-open) or move them to a dedicated window for easy access.
#@markdown #### ℹ️ This notebook uses both our CUDALucas and Prime95 [Bash install scripts](https://github.com/tdulcet/Distributed-Computing-Scripts#organizations) and our [PrimeNet Python script](https://github.com/tdulcet/Distributed-Computing-Scripts#primenet).
#@markdown #### 📜 Please see the [documentation](https://github.com/tdulcet/Distributed-Computing-Scripts/tree/master/google-colab) for more information and to ❤️ support us.
#@markdown #### 🤷 Optionally, create a GIMPS/PrimeNet account [here](https://www.mersenne.org/update/) and [join](https://www.mersenne.org/jteam/) the “Portland State University” team!

prime_ID = 'Default' #@param ['Default'] {allow-input: true}
computer_name = 'Default' #@param ['Default'] {allow-input: true}
GPU_type_of_work = '100 - First time LL tests' #@param ['100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests']
CPU_type_of_work = '150 - First time PRP tests' #@param ['0 - Whatever makes the most sense', '1 - Trial factoring to low limits', '2 - Trial factoring', '4 - P-1 factoring', '5 - ECM for first factors of Mersenne numbers', '6 - ECM on Fermat numbers', '8 - ECM on Mersenne cofactors', '100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests', '150 - First time PRP tests', '151 - Double-check PRP tests', '152 - World record PRP tests', '153 - 100 million digit PRP tests', '154 - First time PRP tests that need P-1 factoring', '155 - Double-check tests using PRP with proof', '160 - First time PRP on Mersenne cofactors', '161 - Double-check PRP on Mersenne cofactors']
CPU_prp_proof_power = '5' #@param ['5', '6', '7', '8', '9', '10', '11', '12']
#@markdown Every lower power halves Drive storage requirements for PRP tests, but doubles the certification cost.
CPU_proof_certification_work = True #@param {type:"boolean"}
computer_number = 'Default (1)' #@param ['Default (1)', '2', '3', '4'] {allow-input: true}
output_type = 'GPU (CUDALucas)' #@param ['GPU and CPU', 'GPU (CUDALucas)', 'CPU (Prime95)'] 
local_time = 'Pacific' #@param ['Pacific', 'Mountain', 'Central', 'Eastern', 'Alaska', 'Hawaii']
debug = 'False' #@param ['False', 'GPU (CUDALucas)', 'CPU (Prime95)']

#@markdown #### 🐛 The *debug* option outputs GPU (CUDALucas) or CPU (Prime95/MPrime) progress and status, then exits.

gpu_info = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this Notebook
path_dir = "" # helps us to not %cd in the optimize_gpu function

import os

class StopExecution(Exception):
  def _render_traceback_(self):
      pass

def optimize_gpu():
  '''If a new GPU is being used, optimize CUDALucas for using this GPU'''
  print('\nOptimizing CUDALucas for this computer and GPU\n')
  !cd cudalucas && chmod 777 CUDALucas
  if not os.path.exists(path_dir + 'cudalucas/' + gpu_info + ' fft.txt'):
    !cd cudalucas && ./CUDALucas -cufftbench 1024 8192 5
  if not os.path.exists(path_dir + 'cudalucas/' + gpu_info + ' threads.txt'):
    !cd cudalucas && ./CUDALucas -threadbench 1024 8192 5 0

def run():
  '''Run CUDALucas and MPrime'''
  print('\nStarting PrimeNet\n')
  !cd cudalucas; nohup python3 primenet.py -d -t 10800 -T $GPU_type_of_work -l "{'local' + computer_number + '.ini'}" >> "{'primenet' + computer_number + '.out'}" &
  !sleep 1
  optimize_gpu()
  while not os.path.exists('cudalucas/worktodo' + computer_number+ '.txt'):
    print(f'Waiting for worktodo{computer_number}.txt access...')
    !sleep 1

  if output_type == 'GPU and CPU':
    print('\nStarting Prime95\n')
    !cd mprime_gpu && chmod 777 mprime; nohup ./mprime -A$computer_number -d >> "{'cpu' + computer_number + '.out'}" &
    print('\nStarting CUDALucas\n')
    !cd cudalucas; nohup ./CUDALucas -i "{'CUDALucas' + computer_number + '.ini'}" >> "{'gpu' + computer_number + '.out'}" &
    !tail -f "{'mprime_gpu/cpu' + computer_number + '.out'}" "{'cudalucas/gpu' + computer_number + '.out'}"
  elif output_type == 'GPU (CUDALucas)':
    print('\nStarting Prime95\n')
    !cd mprime_gpu && chmod 777 mprime; nohup ./mprime -A$computer_number -d >> "{'cpu' + computer_number + '.out'}" &
    print('\nStarting CUDALucas\n')
    !cd cudalucas && ./CUDALucas -k -i "{'CUDALucas' + computer_number + '.ini'}" | tee -a "{'gpu' + computer_number + '.out'}"
  elif output_type == 'CPU (Prime95)':
    print('\nStarting CUDALucas\n')
    !cd cudalucas; nohup ./CUDALucas -i "{'CUDALucas' + computer_number + '.ini'}" >> "{'gpu' + computer_number + '.out'}" &
    print('\nStarting Prime95\n')
    !cd mprime_gpu && chmod 777 mprime && ./mprime -A$computer_number -d | tee -a "{'cpu' + computer_number + '.out'}"

def install():
  '''Download/Install/Configure CUDALucas then Prime95'''
  !wget https://github.com/tdulcet/Distributed-Computing-Scripts/archive/master.zip -nv -O master.zip
  !unzip -o master.zip
  
  print("Downloading, building and setting up CUDALucas\n")
  !cp Distributed-Computing-Scripts-master/{cudalucas2.sh,primenet.py,idletime.sh} .
  !sed -i '/^GPU=/,/^fi/ s/^/# /' cudalucas2.sh # Do not check for an Nvidia GPU
  !sed -i '/^[[:blank:]]*if ! COMPUTE=/,/^[[:blank:]]*fi/!b; /^[[:blank:]]*fi/a echo "$COMPUTE"' cudalucas2.sh # Output CUDA compute capability of GPU
  !sed -i 's/\/$COMPUTE/\/--generate-code arch=compute_37,code=sm_37 --generate-code arch=compute_50,code=sm_50 --generate-code arch=compute_60,code=sm_60 --generate-code arch=compute_70,code=sm_70 --generate-code arch=compute_75,code=sm_75/' cudalucas2.sh
  !sed -i '/^\.\/CUDALucas / s/^/# /' cudalucas2.sh # Disable optimization step for faster install
  !sed -i '/^nohup / s/^/# /' cudalucas2.sh # Do not start PrimeNet
  !sed -i '/^python3 / s/^/# /' cudalucas2.sh # Do not start PrimeNet
  !sed -i '/^crontab / s/^/# /' cudalucas2.sh # Do not create a cronjob
  !bash -- cudalucas2.sh $computer_number $prime_ID $computer_name $GPU_type_of_work
  print("Registering computer with PrimeNet\n")
  !cd cudalucas && python3 primenet.py -d -t 0 -W 1 -T $GPU_type_of_work -u $prime_ID -i "{'worktodo' + computer_number + '.txt'}" -r "{'results' + computer_number + '.txt'}" -l "{'local' + computer_number + '.ini'}" --cudalucas "{'gpu' + computer_number + '.out'}" -H $computer_name
  !cp -u Distributed-Computing-Scripts-master/google-colab/gpu_optimizations/* cudalucas/

  print("Downloading and setting up Prime95\n")
  !cp Distributed-Computing-Scripts-master/{mprime2.sh,mprime2.exp} .
  !sed -i 's/"mprime"/"mprime_gpu"/' mprime2.sh # Name the folder specific to the runtime type
  !sed -i '/^\.\/mprime / s/^/# /' mprime2.sh # Do not start Prime95
  !sed -i '/^nohup / s/^/# /' mprime2.sh # Do not start Prime95
  !sed -i '/^crontab / s/^/# /' mprime2.sh # Do not create a cronjob
  !sed -i '/^expect {/a \\t"Max emergency memory in GB/worker (*):" { sleep 1; send -- "3\\r"; exp_continue }\n\t"Get occasional proof certification work (*):" { sleep 1; send -- "CPU_PROOF_CERTIFICATION_WORK\\r"; exp_continue }' mprime2.exp
  !sed -i 's/CPU_PROOF_CERTIFICATION_WORK/'$CPU_proof_certification_work'/' mprime2.exp
  ![[ -d 'mprime_gpu' ]] && cd mprime_gpu && chmod 777 mprime
  !bash -- mprime2.sh $computer_number $prime_ID $computer_name $CPU_type_of_work # Run script
  file = '"' + f'mprime_gpu/prim{int(computer_number):04d}.txt' + '"'
  !echo 'FixedHardwareUID=1' > temp.txt
  !echo 'ProofPower={CPU_prp_proof_power}' >> temp.txt
  !cat $file >> temp.txt
  !mv temp.txt $file
  !echo 'PreallocateDisk=0' >> '"' + f'mprime_gpu/loca{int(computer_number):04d}.txt' + '"'
  run()

def debug_exit():
  '''Output GPU and output of Prime95 or CUDALucas output'''
  if debug == 'GPU (CUDALucas)'and os.path.exists('cudalucas/gpu' + computer_number + '.out'):
    print(f"\nOutput for computer number {computer_number}:\n")
    print("\nPrimeNet output:\n") 
    !tail -n 100 "{'cudalucas/primenet' + computer_number + '.out'}" # view primenet output
    print("\nGPU (CUDALucas) output: ")
    !tail -n 100 "{'cudalucas/gpu' + computer_number + '.out'}" # view CUDALucas progress
    !cd cudalucas && python3 primenet.py -l "{'local' + computer_number + '.ini'}" -s
    print()
  elif debug == 'CPU (Prime95)' and os.path.exists('mprime_gpu/cpu' + computer_number + '.out'):
    print("\nCPU (Prime95) output:\n") 
    !tail -n 100 "{'mprime_gpu/cpu' + computer_number + '.out'}" # view MPrime progress
    !cd mprime_gpu && chmod 777 mprime && ./mprime -s -A$computer_number
    print()
  else:
    print(f'No `{debug}` output file found for debug option and computer number `{computer_number}`.\n')

def load_drive():
  '''Load & cd into gdrive for persistent data'''
  global path_dir
  if os.path.exists("/content/drive/My Drive"): # create your own notebook with our code
    %cd "/content/drive/My Drive"
    path_dir = "/content/drive/My Drive/GIMPS/"
  else: # use our notebook
    print('Warning: Google Drive is not mounted')
    print('If you were not expecting this, on the far left click the folder icon, the "Mount Drive" folder button, select "CONNECT TO GOOGLE DRIVE" ')
    print('and then re-execute this cell.')
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd "/content/gdrive/My Drive"
    path_dir = "/content/gdrive/My Drive/GIMPS/"
  if "My Drive/GIMPS" in os.getcwd(): # don't create a subfolder in GIMPS/
    return
  !mkdir -p GIMPS
  %cd "GIMPS"

def gpu_check():
  '''GPU Check'''
  global gpu_info
  gpu_info = "\n".join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Select the "Runtime" → "Change runtime type" → "GPU" → "SAVE" to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
    raise StopExecution
  print(f'\nGraphics Processor (GPU):\t{gpu_info}\n')

gpu_check()
!wget https://raw.github.com/tdulcet/Linux-System-Information/master/info.sh -qO - | bash -s # Check System Info
!python3 -V
print()
load_drive()

# set local time
!rm -f /etc/localtime
!ln -s {'/usr/share/zoneinfo/US/' + local_time} /etc/localtime

# use/cleanup input from user
prime_ID = 'psu' if prime_ID.lower() == 'default' else prime_ID
computer_name = "" if computer_name.lower() == 'default' else computer_name
computer_number = "1" if computer_number.lower() == 'default (1)' else computer_number.strip()
CPU_type_of_work = CPU_type_of_work.split("-")[0].rstrip()
CPU_proof_certification_work = "y" if CPU_proof_certification_work else "n"
GPU_type_of_work = GPU_type_of_work.split("-")[0].rstrip()
debug = False if debug == 'False' else debug

# Add quotes to string args so script can parse spaces/special characters
prime_ID = '"' + prime_ID + '"'
computer_name = '"' + computer_name + '"'


if debug:
  debug_exit()
  raise StopExecution

elif not computer_number.isdigit() or int(computer_number) < 0:
  print("ERROR: Computer number must be a number")
  raise StopExecution

elif os.path.exists(f'mprime_gpu/work{int(computer_number):04d}.txt') and os.path.exists('cudalucas/local' + computer_number + '.ini'):
  !cd mprime_gpu && echo -e "$(date)\t$(sed -n 's/^model name[[:space:]]*: *//p' /proc/cpuinfo | uniq)  $(sed -n 's/^model[[:space:]]*: *//p' /proc/cpuinfo | uniq)" >> cpus.txt
  print('\nPrevious CPU counts')
  !cd mprime_gpu; awk -F'\t' '{ print $2 }' cpus.txt | sort | uniq -c | sort -nr
  !cd cudalucas && echo -e "$(date)\t$gpu_info" >> gpus.txt
  print('\nPrevious GPU counts')
  !cd cudalucas; awk -F'\t' '{ print $2 }' gpus.txt | sort | uniq -c | sort -nr
  run()

else:
  install()

print("Gracefully exiting...")